# **Initi**

In [ ]:
!mkdir -p bbmodcache
!curl http://bb-ai.net.s3.amazonaws.com/bb-python-modules/bbSearch.py > bbmodcache/bbSearch.py

from bbmodcache.bbSearch import SearchProblem, search

# possible goal states:
NORMAL_GOAL = [ [1,2,3],
                [4,5,6],
                [7,8,0] ]

SPIRAL_GOAL = [ [1,2,3],
                [8,0,4],
                [7,6,5] ]

# simple
LAYOUT_1 = [ [1,2,3],
             [5,0,6],
             [4,7,8] ] 
# moderate
LAYOUT_2 = [ [1,4,2],
             [6,3,5],
             [0,7,8] ]

# difficult
LAYOUT_3 = [ [8,6,7],
             [2,5,4],
             [3,0,1] ] 

def number_position_in_layout( n, layout):
    for i, row in enumerate(layout):
        for j, val in enumerate(row):
            if val==n:
                return (i,j)

# define the 8-puzzle problem
from copy import deepcopy

class EightPuzzle( SearchProblem ):
        
    action_dict = {
        (0,0) : [(1, 0, 'up'),    (0, 1, 'left')],
        (0,1) : [(0, 0, 'right'), (1, 1, 'up'),    (0, 2, 'left')],
        (0,2) : [(0, 1, 'right'), (1, 2, 'up')],
        (1,0) : [(0, 0, 'down'),  (1, 1, 'left'),  (2, 0, 'up')],
        (1,1) : [(1, 0, 'right'), (0, 1, 'down'),  (1, 2, 'left'), (2, 1, 'up')],
        (1,2) : [(0, 2, 'down'),  (1, 1, 'right'), (2, 2, 'up')],
        (2,0) : [(1, 0, 'down'),  (2, 1, 'left')],
        (2,1) : [(2, 0, 'right'), (1, 1, 'down'),  (2, 2, 'left')],
        (2,2) : [(2, 1, 'right'), (1, 2, 'down')]
    }
    
    
    def __init__(self, initial_layout, goal_layout ):
        pos0 = number_position_in_layout( 0, initial_layout )
        # Initial state is pair giving initial position of space
        # and the initial tile layout.
        self.initial_state = ( pos0, initial_layout)
        self.goal_layout   = goal_layout
        

    ### I just use the position on the board (state[0]) to look up the 
    ### appropriate sequence of possible actions.
    def possible_actions(self, state ):
        actions =  EightPuzzle.action_dict[state[0]]
        actions_with_tile_num = []
        for r, c, d in actions:
            tile_num = state[1][r][c] ## find number of moving tile
            # construct the action representation including the tile number
            actions_with_tile_num.append( (r, c, (tile_num,d)) )
        return actions_with_tile_num

    def successor(self, state, action):
        old0row, old0col  =  state[0]    # get start position
        new0row, new0col, move = action  # unpack the action representation
        moving_number, _ = move
        ## Make a copy of the old layout
        newlayout = deepcopy(state[1])
        # Swap the positions of the new number and the space (rep by 0)
        newlayout[old0row][old0col] = moving_number
        newlayout[new0row][new0col] = 0
        return ((new0row, new0col), newlayout )
    
    def goal_test(self,state):
        return state[1] == self.goal_layout
    
    def display_action(self, action):
        _,_, move = action
        tile_num, direction = move
        print("Move tile", tile_num, direction)
        
    def display_state(self,state):
        for row in state[1]:
            nums = [ (n if n>0 else '.') for n in row]
            print( "   ", nums[0], nums[1], nums[2] )

    def my_misplaced_tiles(self, state):
        count = 0
        for i in range(3):
            for j in range(0, 3):
              if state[1][i][j] != self.goal_layout[i][j]:
                count += 1
        return count

    def my_manhattan(self, state):
        count = 0
        for i in range(0, 3):
            for j in range(3):
                val = state[1][i][j]
                x, y = number_position_in_layout(val, self.goal_layout)
                count += abs(x - i)
                count += abs(y - j)
        return count
            
# simple puzzle
EP1 = EightPuzzle( LAYOUT_1, NORMAL_GOAL )
# moderate puzzle
EP2 = EightPuzzle( LAYOUT_2, NORMAL_GOAL )
# difficult puzzle
EP3 = EightPuzzle( LAYOUT_3, NORMAL_GOAL )

!mkdir -p bbmodcache
!curl http://bb-ai.net.s3.amazonaws.com/bb-python-modules/crazy8heuristics.py > bbmodcache/crazy8heuristics.py
from bbmodcache.crazy8heuristics import bb_misplaced_tiles, bb_manhattan


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18767  100 18767    0     0  40533      0 --:--:-- --:--:-- --:--:-- 40533
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4731  100  4731    0     0  15822      0 --:--:-- --:--:-- --:--:-- 15770


# **Simple puzzle**

In [ ]:
# BFS loop check
print("********************************BFS LOOP CHECK********************************")
T1_1=search(EP1, 'BF/FIFO', 1000, loop_check=True, return_info=False)
print("******************************************************************************")
# BFS no loop check
print("********************************BFS NO LOOP CHECK********************************")
T1_2=search(EP1, 'BF/FIFO', 1000, return_info=False)
print("*********************************************************************************")

********************************BFS LOOP CHECK********************************
This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=None
Max search nodes: 1000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)

:-)) *SUCCESS* ((-:

Path length = 4
Goal state is:
    1 2 3
    4 5 6
    7 8 .
The action path to the solution is:
Move tile 5 right
Move tile 4 up
Move tile 7 left
Move tile 8 left


SEARCH SPACE STATS:
Total nodes generated          =       67  (includes start)
Nodes discarded by loop_check  =       23  (44 distinct states added to queue)
Nodes tested (by goal_test)    =       25  (24 expanded + 1 goal)
Nodes left in queue            =       19

Time taken = 0.0125 seconds

******************************************************************************
********************************BFS NO LOOP CHECK**

In [ ]:
# it's fast, but too many steps
# DFS loop check fix
print("********************************DFS LOOP CHECK FIX********************************")
T1_3=search(EP1, 'DF/LIFO', 1000, loop_check=True, return_info=False)
print("*************************************************************************************")

********************************DFS LOOP CHECK FIX********************************
This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=DF/LIFO, cost=None, heuristic=None
Max search nodes: 1000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)

:-)) *SUCCESS* ((-:

Path length = 58
Goal state is:
    1 2 3
    4 5 6
    7 8 .
The action path to the solution is:
Move tile 7 up
Move tile 8 left
Move tile 6 down
Move tile 7 right
Move tile 8 up
Move tile 6 left
Move tile 7 down
Move tile 8 right
Move tile 6 up
Move tile 4 right
Move tile 5 down
Move tile 6 left
Move tile 4 up
Move tile 7 left
Move tile 8 down
Move tile 4 right
Move tile 7 up
Move tile 8 left
Move tile 4 down
Move tile 7 right
Move tile 8 up
Move tile 5 right
Move tile 6 down
Move tile 8 left
Move tile 5 up
Move tile 4 left
Move tile 7 down
Move tile 5 right
Move tile 4 up
Move ti

In [ ]:
# Sometimes, it's fast with few steps. In the other time, it would fail. 
# Furthermroe, there are many steps with slow speed
# DFS loop check random
print("********************************DFS LOOP CHECK RANDOM********************************")
T1_4=search(EP1, 'DF/LIFO', 100000, loop_check=True, randomise=True, return_info=False)
print("****************************************************************************************")

Streaming output truncated to the last 5000 lines.
Move tile 6 up
Move tile 2 left
Move tile 4 left
Move tile 5 down
Move tile 7 down
Move tile 3 right
Move tile 1 up
Move tile 7 left
Move tile 5 up
Move tile 4 right
Move tile 7 down
Move tile 1 down
Move tile 8 right
Move tile 6 up
Move tile 1 left
Move tile 5 left
Move tile 4 up
Move tile 7 right
Move tile 5 down
Move tile 8 down
Move tile 3 left
Move tile 4 up
Move tile 7 up
Move tile 5 right
Move tile 2 right
Move tile 1 down
Move tile 6 down
Move tile 3 left
Move tile 8 up
Move tile 6 right
Move tile 1 up
Move tile 2 left
Move tile 5 left
Move tile 7 down
Move tile 6 right
Move tile 8 down
Move tile 4 left
Move tile 6 up
Move tile 8 right
Move tile 4 down
Move tile 3 right
Move tile 1 up
Move tile 2 up
Move tile 5 left
Move tile 7 left
Move tile 8 down
Move tile 6 down
Move tile 3 right
Move tile 1 right
Move tile 2 up
Move tile 4 left
Move tile 6 left
Move tile 8 up
Move tile 7 right
Move tile 5 right
Move tile 4 down
Move tile 6

In [ ]:
# No result. RAM not enough
# DFS no loop check fix
print("********************************DFS NO LOOP CHECK FIX********************************")
T1_5=search(EP1, 'DF/LIFO', 60000, return_info=False)
print("*****************************************************************************************")

********************************DFS NO LOOP CHECK FIX********************************
This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=DF/LIFO, cost=None, heuristic=None
Max search nodes: 60000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
........................
!! Search node limit (60000) reached !!
): No solution found :(


SEARCH SPACE STATS:
Total nodes generated          =    60001  (includes start)
Nodes tested (by goal_test)    =    24001  (all expanded)
Nodes left in queue            =    35999

Time taken = 56.9661 seconds

*****************************************************************************************


In [ ]:
# Slow... with no result
# DFS no loop check random
print("********************************DFS NO LOOP CHECK RANDOM********************************")
T1_6=search(EP1, 'DF/LIFO', 100000, randomise=True)
print("********************************************************************************************")

********************************DFS NO LOOP CHECK RANDOM********************************
This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=DF/LIFO, cost=None, heuristic=None
Max search nodes: 120000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
........................................

In [ ]:
!mkdir -p bbmodcache
!curl http://bb-ai.net.s3.amazonaws.com/bb-python-modules/crazy8heuristics.py > bbmodcache/crazy8heuristics.py
from bbmodcache.crazy8heuristics import bb_misplaced_tiles, bb_manhattan

# best-first using misplaced tiles
print("********************************BEST FIRST MISPLACED FILES********************************")
T1_7=search(EP1, 'BF/FIFO', 5000, heuristic=bb_misplaced_tiles, loop_check=True, return_info=False)
print("**********************************************************************************************")

# best-first using misplaced tiles
print("********************************BEST FIRST MANHATTAN********************************")
T1_8=search(EP1, 'BF/FIFO', 5000, heuristic=bb_manhattan, loop_check=True, return_info=False)
print("**************************************************************************************")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4731  100  4731    0     0  15929      0 --:--:-- --:--:-- --:--:-- 15929
********************************BEST FIRST MISPLACED FILES********************************
This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=bb_misplaced_tiles
Max search nodes: 5000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)

:-)) *SUCCESS* ((-:

Path length = 4
Goal state is:
    1 2 3
    4 5 6
    7 8 .
The action path to the solution is:
Move tile 5 right
Move tile 4 up
Move tile 7 left
Move tile 8 left


SEARCH SPACE STATS:
Total nodes generated          =       13  (includes start)
Nodes discarded by loop_check  =        3  (10 distinct states added to queue)
Nodes

In [ ]:
def cost(path, state):
  return len(path)

# A*
print("********************************A* MISPLACED FILES********************************")
T1_9=search(EP1, 'BF/FIFO', 5000, cost=cost, heuristic=bb_misplaced_tiles, loop_check=True, return_info=False)
print("************************************************************************************")
print("********************************A* MANHATTAN********************************")
T1_10=search(EP1, 'BF/FIFO', 5000, cost=cost, heuristic=bb_manhattan, loop_check=True, return_info=False)
print("************************************************************************************")

********************************A* MISPLACED FILES********************************
This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=cost, heuristic=bb_misplaced_tiles
Max search nodes: 5000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)

:-)) *SUCCESS* ((-:

Path length = 4
Goal state is:
    1 2 3
    4 5 6
    7 8 .
Cost of reaching goal: 4
The action path to the solution is:
Move tile 5 right
Move tile 4 up
Move tile 7 left
Move tile 8 left


SEARCH SPACE STATS:
Total nodes generated          =       13  (includes start)
Nodes discarded by loop_check  =        3  (10 distinct states added to queue)
Nodes tested (by goal_test)    =        5  (4 expanded + 1 goal)
Nodes left in queue            =        5

Time taken = 0.0057 seconds

************************************************************************************
***

# **Simple result show**

In [ ]:
# T1_1=search(EP1, 'BF/FIFO', 1000, loop_check=True, return_info=True)
# T1_2=search(EP1, 'BF/FIFO', 1000, return_info=True)
# T1_3=search(EP1, 'DF/LIFO', 1000, loop_check=True, return_info=True)
# # T1_4=search(EP1, 'DF/LIFO', 100000, loop_check=True, randomise=True, return_info=True)
# # T1_5=search(EP1, 'DF/LIFO', 60000, return_info=True)
# # T1_6=search(EP1, 'DF/LIFO', 100000, randomise=True, return_info=True)
# !mkdir -p bbmodcache
# !curl http://bb-ai.net.s3.amazonaws.com/bb-python-modules/crazy8heuristics.py > bbmodcache/crazy8heuristics.py
# from bbmodcache.crazy8heuristics import bb_misplaced_tiles, bb_manhattan
# T1_7=search(EP1, 'BF/FIFO', 100, heuristic=bb_misplaced_tiles, loop_check=True, return_info=True)
# T1_8=search(EP1, 'BF/FIFO', 100, heuristic=bb_manhattan, loop_check=True, return_info=True)
# def cost(path, state):
#   return len(path)
# T1_9=search(EP1, 'BF/FIFO', 100, cost=cost, heuristic=bb_misplaced_tiles, loop_check=True, return_info=True)
# T1_10=search(EP1, 'BF/FIFO', 100, cost=cost, heuristic=bb_manhattan, loop_check=True, return_info=True)


def cost(path, state):
  return len(path)
T1_1=search(EP1, 'BF/FIFO', 100, cost=cost, heuristic=bb_misplaced_tiles, loop_check=True, return_info=True)
T1_2=search(EP1, 'BF/FIFO', 100, cost=cost, heuristic=EP1.my_misplaced_tiles, loop_check=True, return_info=True)
T1_3=search(EP1, 'BF/FIFO', 100, cost=cost, heuristic=bb_manhattan, loop_check=True, return_info=True)
T1_4=search(EP1, 'BF/FIFO', 100, cost=cost, heuristic=EP1.my_manhattan, loop_check=True, return_info=True)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=cost, heuristic=bb_misplaced_tiles
Max search nodes: 100  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)

:-)) *SUCCESS* ((-:

Path length = 4
Goal state is:
    1 2 3
    4 5 6
    7 8 .
Cost of reaching goal: 4
The action path to the solution is:
Move tile 5 right
Move tile 4 up
Move tile 7 left
Move tile 8 left


SEARCH SPACE STATS:
Total nodes generated          =       13  (includes start)
Nodes discarded by loop_check  =        3  (10 distinct states added to queue)
Nodes tested (by goal_test)    =        5  (4 expanded + 1 goal)
Nodes left in queue            =        5

Time taken = 0.0095 seconds

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF

In [ ]:
AL_NAME = ['BFS_LT', 'BFS_LF', 'DFS_LT_RF', 'DFS_LT_RT']
TEST_RESULTS ={'BFS_LT': T1_1, 
               'BFS_LF': T1_2, 
               'DFS_LT_RF': T1_3, 
               'DFS_LT_RT': T1_4, 
               }

short_tc = {"GOAL_STATE_FOUND"     : "Y",
            "NODE_LIMIT_EXCEEDED"  : "!",
            "SEARH-SPACE_EXHAUSTED": "x"}

print("\n                **TESTS SUMMARY**\n")

print("     Test        #max   Result   #gen      #inQ     Time s  cost")
for i, test in enumerate(TEST_RESULTS):
    max  = TEST_RESULTS[test]['args']['max_nodes']
    tc  = TEST_RESULTS[test]['result']['termination_condition']
    stc = short_tc[tc]
    
    ng  = TEST_RESULTS[test]['search_stats']['nodes_generated']
    nq  = TEST_RESULTS[test]['search_stats']['nodes_left_in_queue']
    time = round( TEST_RESULTS[test]['search_stats']['time_taken'], 2 )

    path = TEST_RESULTS[test]['result']['path']
    if isinstance(path, list):
      cost = len(path)
    else:
      cost = '!'
    print( f"{test:>10}:   {max:>8}    {stc}  {ng:>8} {nq:>8}     {time:>6}    {cost}")



                **TESTS SUMMARY**

     Test        #max   Result   #gen      #inQ     Time s  cost
    BFS_LT:        100    Y        13        5       0.01    4
    BFS_LF:        100    Y        13        5        0.0    4
 DFS_LT_RF:        100    Y        13        5        0.0    4
 DFS_LT_RT:        100    Y        13        5        0.0    4


# **Moderate puzzle**

In [ ]:
# BFS loop check
print("********************************BFS LOOP CHECK********************************")
T2_1=search(EP2, 'BF/FIFO', 40000, loop_check=True)
print("******************************************************************************")
# BFS no loop check
print("********************************BFS NO LOOP CHECK********************************")
T2_2=search(EP2, 'BF/FIFO', 1400000)
print("*********************************************************************************")

********************************BFS LOOP CHECK********************************
******************************************************************************
********************************BFS NO LOOP CHECK********************************
This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=None
Max search nodes: 1400000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
.................................................................................................... (100000)
.................................................................................................... (200000)
.................................................................................................... (300000)
.................................................................................................... (400000)
...........

In [ ]:
# !!!! long time !!!!
# DFS loop check fix
print("********************************DFS LOOP CHECK FIX********************************")
T2_3=search(EP2, 'DF/LIFO', 80000, loop_check=True)
print("*************************************************************************************")

Streaming output truncated to the last 5000 lines.
Move tile 8 left
Move tile 1 down
Move tile 4 right
Move tile 8 up
Move tile 7 right
Move tile 6 down
Move tile 8 left
Move tile 7 up
Move tile 1 left
Move tile 4 down
Move tile 7 right
Move tile 1 up
Move tile 4 left
Move tile 7 down
Move tile 1 right
Move tile 4 up
Move tile 6 right
Move tile 8 down
Move tile 4 left
Move tile 6 up
Move tile 7 left
Move tile 1 down
Move tile 6 right
Move tile 7 up
Move tile 1 left
Move tile 6 down
Move tile 7 right
Move tile 3 down
Move tile 2 left
Move tile 7 up
Move tile 6 up
Move tile 1 right
Move tile 3 down
Move tile 6 left
Move tile 1 up
Move tile 3 right
Move tile 6 down
Move tile 1 left
Move tile 3 up
Move tile 6 right
Move tile 8 right
Move tile 4 down
Move tile 5 down
Move tile 2 left
Move tile 7 left
Move tile 3 up
Move tile 6 up
Move tile 8 right
Move tile 1 down
Move tile 6 left
Move tile 8 up
Move tile 1 right
Move tile 6 down
Move tile 8 left
Move tile 3 down
Move tile 7 right
Move tile

In [ ]:
# !!! little long time !!!
# DFS loop check random
print("********************************DFS LOOP CHECK RANDOM********************************")
T2_4=search(EP2, 'DF/LIFO', 120000, loop_check=True, randomise=True)
print("****************************************************************************************")

Streaming output truncated to the last 5000 lines.
Move tile 2 down
Move tile 8 left
Move tile 5 left
Move tile 4 up
Move tile 3 right
Move tile 6 up
Move tile 7 left
Move tile 3 down
Move tile 4 down
Move tile 5 right
Move tile 8 right
Move tile 2 up
Move tile 6 left
Move tile 7 up
Move tile 3 left
Move tile 4 down
Move tile 5 down
Move tile 8 right
Move tile 2 right
Move tile 6 up
Move tile 7 left
Move tile 2 down
Move tile 8 left
Move tile 5 up
Move tile 2 right
Move tile 3 up
Move tile 4 left
Move tile 2 down
Move tile 3 right
Move tile 4 up
Move tile 1 right
Move tile 7 down
Move tile 4 left
Move tile 8 down
Move tile 5 left
Move tile 3 up
Move tile 8 right
Move tile 4 right
Move tile 6 down
Move tile 5 left
Move tile 4 up
Move tile 8 left
Move tile 3 down
Move tile 4 right
Move tile 8 up
Move tile 1 up
Move tile 2 left
Move tile 3 down
Move tile 1 right
Move tile 2 up
Move tile 3 left
Move tile 1 down
Move tile 4 down
Move tile 8 right
Move tile 5 right
Move tile 6 up
Move tile 2

In [ ]:
# DFS no loop check fix
print("********************************DFS NO LOOP CHECK FIX********************************")
T2_5=search(EP2, 'DF/LIFO', 100000)
print("*****************************************************************************************")

********************************DFS NO LOOP CHECK FIX********************************
This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=DF/LIFO, cost=None, heuristic=None
Max search nodes: 100000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
........................................
!! Search node limit (100000) reached !!
): No solution found :(


SEARCH SPACE STATS:
Total nodes generated          =   100001  (includes start)
Nodes tested (by goal_test)    =    40001  (all expanded)
Nodes left in queue            =    59999

Time taken = 130.409 seconds

*****************************************************************************************


In [ ]:
# DFS no loop check random
print("********************************DFS NO LOOP CHECK RANDOM********************************")
T2_6=search(EP2, 'DF/LIFO', 120000, randomise=True)
print("********************************************************************************************")

********************************DFS NO LOOP CHECK RANDOM********************************
This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=DF/LIFO, cost=None, heuristic=None
Max search nodes: 120000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
........................................

In [ ]:
# best-first using misplaced tiles
print("********************************BEST FIRST MISPLACED FILES********************************")
T2_7=search(EP2, 'BF/FIFO', 300, heuristic=bb_misplaced_tiles, loop_check=True)
print("**********************************************************************************************")

# best-first using misplaced tiles
print("********************************BEST FIRST MANHATTAN********************************")
T2_8=search(EP2, 'BF/FIFO', 200, heuristic=bb_manhattan, loop_check=True)
print("**************************************************************************************")

********************************BEST FIRST MISPLACED FILES********************************
This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=bb_misplaced_tiles
Max search nodes: 300  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)

:-)) *SUCCESS* ((-:

Path length = 34
Goal state is:
    1 2 3
    4 5 6
    7 8 .
The action path to the solution is:
Move tile 7 left
Move tile 8 left
Move tile 5 down
Move tile 3 right
Move tile 4 down
Move tile 2 left
Move tile 3 up
Move tile 5 up
Move tile 8 right
Move tile 4 down
Move tile 6 right
Move tile 7 up
Move tile 4 left
Move tile 8 left
Move tile 5 down
Move tile 6 right
Move tile 7 right
Move tile 4 up
Move tile 8 left
Move tile 5 left
Move tile 6 down
Move tile 7 right
Move tile 5 up
Move tile 8 right
Move tile 4 down
Move tile 5 left
Move tile 7 left
Move tile 6 up

In [ ]:
def cost(path, state):
  return len(path)

# A*
print("********************************A* MISPLACED FILES********************************")
T2_9=search(EP2, 'BF/FIFO', 3000, cost=cost, heuristic=bb_misplaced_tiles, loop_check=True)
print("************************************************************************************")
print("********************************A* MANHATTAN********************************")
T2_10=search(EP2, 'BF/FIFO', 500, cost=cost, heuristic=bb_manhattan, loop_check=True)
print("************************************************************************************")

********************************A* MISPLACED FILES********************************
This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=cost, heuristic=bb_misplaced_tiles
Max search nodes: 3000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
.
:-)) *SUCCESS* ((-:

Path length = 18
Goal state is:
    1 2 3
    4 5 6
    7 8 .
Cost of reaching goal: 18
The action path to the solution is:
Move tile 7 left
Move tile 8 left
Move tile 5 down
Move tile 3 right
Move tile 6 right
Move tile 7 up
Move tile 8 left
Move tile 6 down
Move tile 4 down
Move tile 2 left
Move tile 3 up
Move tile 5 up
Move tile 6 right
Move tile 8 right
Move tile 7 down
Move tile 4 left
Move tile 5 left
Move tile 6 up


SEARCH SPACE STATS:
Total nodes generated          =     4045  (includes start)
Nodes discarded by loop_check  =     1628  (2417 distinct states a

# **Moderate result show**

In [ ]:
T2_1=search(EP2, 'BF/FIFO', 40000, loop_check=True, return_info=True)
T2_2=search(EP2, 'BF/FIFO', 1400000, return_info=True)
T2_3=search(EP2, 'DF/LIFO', 80000, loop_check=True, return_info=True)
T2_4=search(EP2, 'DF/LIFO', 120000, loop_check=True, randomise=True, return_info=True)
T2_5=search(EP2, 'DF/LIFO', 100000, return_info=True)
T2_6=search(EP2, 'DF/LIFO', 100000, randomise=True, return_info=True)
!mkdir -p bbmodcache
!curl http://bb-ai.net.s3.amazonaws.com/bb-python-modules/crazy8heuristics.py > bbmodcache/crazy8heuristics.py
from bbmodcache.crazy8heuristics import bb_misplaced_tiles, bb_manhattan
T2_7=search(EP2, 'BF/FIFO', 300, heuristic=bb_misplaced_tiles, loop_check=True, return_info=True)
T2_8=search(EP2, 'BF/FIFO', 200, heuristic=bb_manhattan, loop_check=True, return_info=True)
def cost(path, state):
  return len(path)
T2_9=search(EP2, 'BF/FIFO', 3000, cost=cost, heuristic=bb_misplaced_tiles, loop_check=True, return_info=True)
T2_10=search(EP2, 'BF/FIFO', 500, cost=cost, heuristic=bb_manhattan, loop_check=True, return_info=True)

Streaming output truncated to the last 5000 lines.
Move tile 1 left
Move tile 5 up
Move tile 2 left
Move tile 6 down
Move tile 5 right
Move tile 2 up
Move tile 6 left
Move tile 5 down
Move tile 2 right
Move tile 6 up
Move tile 7 right
Move tile 1 down
Move tile 6 left
Move tile 7 up
Move tile 5 left
Move tile 2 down
Move tile 7 right
Move tile 5 up
Move tile 2 left
Move tile 7 down
Move tile 5 right
Move tile 2 up
Move tile 1 right
Move tile 6 down
Move tile 2 left
Move tile 1 up
Move tile 7 left
Move tile 5 down
Move tile 1 right
Move tile 7 up
Move tile 5 left
Move tile 1 down
Move tile 7 right
Move tile 5 up
Move tile 6 right
Move tile 2 down
Move tile 5 left
Move tile 6 up
Move tile 1 left
Move tile 7 down
Move tile 6 right
Move tile 1 up
Move tile 7 left
Move tile 6 down
Move tile 1 right
Move tile 5 right
Move tile 2 up
Move tile 7 left
Move tile 6 left
Move tile 1 down
Move tile 5 right
Move tile 6 up
Move tile 1 left
Move tile 5 down
Move tile 6 right
Move tile 3 down
Move tile

In [ ]:
AL_NAME = ['BFS_LT', 'BFS_LF', 'DFS_LT_RF', 'DFS_LT_RT', 'DFS_LF_RF', 'DFS_LF_RT', 'h1', 'h2', 'h1A*', 'h2A*']
TEST_RESULTS ={'BFS_LT': T2_1, 
               'BFS_LF': T2_2, 
               'DFS_LT_RF': T2_3, 
               'DFS_LT_RT': T2_4, 
               'DFS_LF_RF': T2_5, 
               'DFS_LF_RT': T2_6, 
               'h1': T2_7, 
               'h2': T2_8, 
               'h1A*': T2_9, 
               'h2A*': T2_10}

short_tc = {"GOAL_STATE_FOUND"     : "Y",
            "NODE_LIMIT_EXCEEDED"  : "!",
            "SEARH-SPACE_EXHAUSTED": "x"}

print("\n                **TESTS SUMMARY**\n")

print("     Test        #max   Result   #gen      #inQ     Time s  cost")
for i, test in enumerate(TEST_RESULTS):
    max  = TEST_RESULTS[test]['args']['max_nodes']
    tc  = TEST_RESULTS[test]['result']['termination_condition']
    stc = short_tc[tc]
    
    ng  = TEST_RESULTS[test]['search_stats']['nodes_generated']
    nq  = TEST_RESULTS[test]['search_stats']['nodes_left_in_queue']
    time = round( TEST_RESULTS[test]['search_stats']['time_taken'], 2 )

    path = TEST_RESULTS[test]['result']['path']
    if isinstance(path, list):
      cost = len(path)
    else:
      cost = '!'
    # cost = len(path)
    print( f"{test:>10}:   {max:>8}    {stc}  {ng:>8} {nq:>8}     {time:>6}    {cost}")



                **TESTS SUMMARY**

     Test        #max   Result   #gen      #inQ     Time s  cost
    BFS_LT:      40000    Y     70191    10814      56.65    18
    BFS_LF:    1400000    !   1400001   908366      126.7    !
 DFS_LT_RF:      80000    Y    122275    32712      80.31    37836
 DFS_LT_RT:     120000    !    213597    42531     192.76    !
 DFS_LF_RF:     100000    !    100001    59999     176.15    !
 DFS_LF_RT:     100000    !    100001    64695     199.71    !
        h1:        300    Y       396       98       0.58    34
        h2:        200    Y       253       61       0.02    34
      h1A*:       3000    Y      4045      936       0.18    18
      h2A*:        500    Y       819      181       0.03    18


# **Difficult puzzle**

In [ ]:
# BFS loop check no
print("********************************BFS LOOP CHECK********************************")
T3_1=search(EP3, 'BF/FIFO', 180000, loop_check=True)
print("******************************************************************************")

********************************BFS LOOP CHECK********************************
This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=None
Max search nodes: 180000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
.................................................................................................... (100000)
...........................................................................
!! Search node limit (180000) reached !!
): No solution found :(


SEARCH SPACE STATS:
Total nodes generated          =   469739  (includes start)
Nodes discarded by loop_check  =   289738  (180001 distinct states added to queue)
Nodes tested (by goal_test)    =   175913  (all expanded)
Nodes left in queue            =     4087

Time taken = 11.4922 seconds

******************************************************************

In [ ]:
# BFS no loop check yes
print("********************************BFS NO LOOP CHECK********************************")
T3_2=search(EP3, 'BF/FIFO', 2000000)
print("*********************************************************************************")

********************************BFS NO LOOP CHECK********************************
This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=None
Max search nodes: 2000000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
.................................................................................................... (100000)
.................................................................................................... (200000)
.................................................................................................... (300000)
.................................................................................................... (400000)
.................................................................................................... (500000)
...........................................................

In [ ]:
# DFS loop check fix yes
print("********************************DFS LOOP CHECK FIX********************************")
T3_3=search(EP3, 'DF/LIFO', 14000, loop_check=True)
print("*************************************************************************************")

Streaming output truncated to the last 5000 lines.
Move tile 1 right
Move tile 7 up
Move tile 3 right
Move tile 2 down
Move tile 7 left
Move tile 3 up
Move tile 5 left
Move tile 1 down
Move tile 3 right
Move tile 5 up
Move tile 1 left
Move tile 3 down
Move tile 5 right
Move tile 6 down
Move tile 8 left
Move tile 5 up
Move tile 3 up
Move tile 1 right
Move tile 6 down
Move tile 3 left
Move tile 1 up
Move tile 6 right
Move tile 3 down
Move tile 1 left
Move tile 5 down
Move tile 8 right
Move tile 1 up
Move tile 3 up
Move tile 6 left
Move tile 5 down
Move tile 3 right
Move tile 6 up
Move tile 5 left
Move tile 3 down
Move tile 6 right
Move tile 5 up
Move tile 2 right
Move tile 7 down
Move tile 5 left
Move tile 2 up
Move tile 3 left
Move tile 6 down
Move tile 2 right
Move tile 3 up
Move tile 6 left
Move tile 2 down
Move tile 3 right
Move tile 6 up
Move tile 7 right
Move tile 5 down
Move tile 6 left
Move tile 7 up
Move tile 2 left
Move tile 3 down
Move tile 7 right
Move tile 2 up
Move tile 3 l

In [ ]:
# DFS loop check random yes
print("********************************DFS LOOP CHECK RANDOM********************************")
T3_4=search(EP3, 'DF/LIFO', 80000, loop_check=True, randomise=True)
print("****************************************************************************************")

********************************DFS LOOP CHECK RANDOM********************************
This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=DF/LIFO, cost=None, heuristic=None
Max search nodes: 80000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
...............................................
!! Search node limit (80000) reached !!
): No solution found :(


SEARCH SPACE STATS:
Total nodes generated          =   133815  (includes start)
Nodes discarded by loop_check  =    53814  (80001 distinct states added to queue)
Nodes tested (by goal_test)    =    47894  (all expanded)
Nodes left in queue            =    32106

Time taken = 68.5095 seconds

****************************************************************************************


In [ ]:
# DFS no loop check fix yes
print("********************************DFS NO LOOP CHECK FIX********************************")
T3_5=search(EP3, 'DF/LIFO', 50000)
print("*****************************************************************************************")

********************************DFS NO LOOP CHECK FIX********************************
This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=DF/LIFO, cost=None, heuristic=None
Max search nodes: 50000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
....................
!! Search node limit (50000) reached !!
): No solution found :(


SEARCH SPACE STATS:
Total nodes generated          =    50001  (includes start)
Nodes tested (by goal_test)    =    20001  (all expanded)
Nodes left in queue            =    29999

Time taken = 25.441 seconds

*****************************************************************************************


In [ ]:
# DFS no loop check random yes
print("********************************DFS NO LOOP CHECK RANDOM********************************")
T3_6=search(EP3, 'DF/LIFO', 50000, randomise=True)
print("********************************************************************************************")

********************************DFS NO LOOP CHECK RANDOM********************************
This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=DF/LIFO, cost=None, heuristic=None
Max search nodes: 50000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
.................
!! Search node limit (50000) reached !!
): No solution found :(


SEARCH SPACE STATS:
Total nodes generated          =    50001  (includes start)
Nodes tested (by goal_test)    =    17650  (all expanded)
Nodes left in queue            =    32350

Time taken = 22.0066 seconds

********************************************************************************************


In [ ]:
# best-first using misplaced tiles
print("********************************BEST FIRST MISPLACED FILES********************************")
T3_7=search(EP3, 'BF/FIFO', 1000, heuristic=bb_misplaced_tiles, loop_check=True)
print("**********************************************************************************************")

# best-first using manhattan distance
print("********************************BEST FIRST MANHATTAN********************************")
T3_8=search(EP3, 'BF/FIFO', 200, heuristic=bb_manhattan, loop_check=True)
print("**************************************************************************************")

********************************BEST FIRST MISPLACED FILES********************************
This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=bb_misplaced_tiles
Max search nodes: 1000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)

:-)) *SUCCESS* ((-:

Path length = 63
Goal state is:
    1 2 3
    4 5 6
    7 8 .
The action path to the solution is:
Move tile 1 left
Move tile 4 down
Move tile 7 down
Move tile 6 right
Move tile 8 right
Move tile 2 up
Move tile 3 up
Move tile 1 left
Move tile 4 left
Move tile 7 down
Move tile 6 down
Move tile 8 right
Move tile 2 right
Move tile 3 up
Move tile 1 up
Move tile 4 left
Move tile 5 down
Move tile 2 down
Move tile 3 right
Move tile 1 up
Move tile 4 up
Move tile 5 left
Move tile 7 left
Move tile 6 down
Move tile 8 down
Move tile 3 right
Move tile 2 up
Move tile 8 left
M

In [ ]:
def cost(path, state):
  return len(path)

# A*
print("********************************A* MISPLACED FILES********************************")
T3_9=search(EP3, 'BF/FIFO', 200000, cost=cost, heuristic=bb_misplaced_tiles, loop_check=True)
print("************************************************************************************")
print("********************************A* MANHATTAN********************************")
T3_10=search(EP3, 'BF/FIFO', 30000, cost=cost, heuristic=bb_manhattan, loop_check=True)
print("************************************************************************************")

********************************A* MISPLACED FILES********************************
This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=cost, heuristic=bb_misplaced_tiles
Max search nodes: 200000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
.................................................................................................... (100000)
...........................................
:-)) *SUCCESS* ((-:

Path length = 31
Goal state is:
    1 2 3
    4 5 6
    7 8 .
Cost of reaching goal: 31
The action path to the solution is:
Move tile 1 left
Move tile 4 down
Move tile 7 down
Move tile 6 right
Move tile 8 right
Move tile 2 up
Move tile 3 up
Move tile 1 left
Move tile 4 left
Move tile 7 down
Move tile 5 right
Move tile 3 right
Move tile 1 up
Move tile 4 left
Move tile 7 left
Move tile 5 down
Move tile 3 right
Move til

# **Difficult result show**

In [ ]:
T3_1=search(EP3, 'BF/FIFO', 180000, loop_check=True, return_info=True)
T3_2=search(EP3, 'BF/FIFO', 2000000, return_info=True)
T3_3=search(EP3, 'DF/LIFO', 14000, loop_check=True, return_info=True)
T3_4=search(EP3, 'DF/LIFO', 80000, loop_check=True, randomise=True, return_info=True)
T3_5=search(EP3, 'DF/LIFO', 50000, return_info=True)
T3_6=search(EP3, 'DF/LIFO', 50000, randomise=True, return_info=True)
!mkdir -p bbmodcache
!curl http://bb-ai.net.s3.amazonaws.com/bb-python-modules/crazy8heuristics.py > bbmodcache/crazy8heuristics.py
from bbmodcache.crazy8heuristics import bb_misplaced_tiles, bb_manhattan
T3_7=search(EP3, 'BF/FIFO', 1000, heuristic=bb_misplaced_tiles, loop_check=True, return_info=True)
T3_8=search(EP3, 'BF/FIFO', 200, heuristic=bb_manhattan, loop_check=True, return_info=True)
def cost(path, state):
  return len(path)
T3_9=search(EP3, 'BF/FIFO', 200000, cost=cost, heuristic=bb_misplaced_tiles, loop_check=True, return_info=True)
T3_10=search(EP3, 'BF/FIFO', 30000, cost=cost, heuristic=bb_manhattan, loop_check=True, return_info=True)

Streaming output truncated to the last 5000 lines.
Move tile 8 right
Move tile 5 up
Move tile 5 down
Move tile 5 up
Move tile 3 left
Move tile 7 left
Move tile 6 up
Move tile 6 down
Move tile 7 right
Move tile 1 up
Move tile 4 right
Move tile 4 left
Move tile 4 right
Move tile 3 down
Move tile 1 left
Move tile 1 right
Move tile 3 up
Move tile 4 left
Move tile 1 down
Move tile 7 left
Move tile 2 down
Move tile 8 right
Move tile 7 up
Move tile 3 right
Move tile 3 left
Move tile 7 down
Move tile 5 right
Move tile 5 left
Move tile 5 right
Move tile 3 up
Move tile 3 down
Move tile 5 left
Move tile 5 right
Move tile 5 left
Move tile 8 left
Move tile 8 right
Move tile 7 up
Move tile 2 left
Move tile 2 right
Move tile 7 down
Move tile 8 left
Move tile 8 right
Move tile 5 right
Move tile 5 left
Move tile 5 right
Move tile 5 left
Move tile 8 left
Move tile 2 up
Move tile 7 right
Move tile 3 right
Move tile 4 up
Move tile 1 left
Move tile 6 left
Move tile 7 down
Move tile 3 right
Move tile 3 left

# **Summary**

In [ ]:
AL_NAME = ['BFS_LT', 'BFS_LF', 'DFS_LT_RF', 'DFS_LT_RT', 'DFS_LF_RF', 'DFS_LF_RT', 'h1', 'h2', 'h1A*', 'h2A*']
TEST_RESULTS ={'BFS_LT': T3_1, 
               'BFS_LF': T3_2, 
               'DFS_LT_RF': T3_3, 
               'DFS_LT_RT': T3_4, 
               'DFS_LF_RF': T3_5, 
               'DFS_LF_RT': T3_6, 
               'h1': T3_7, 
               'h2': T3_8, 
               'h1A*': T3_9, 
               'h2A*': T3_10}

short_tc = {"GOAL_STATE_FOUND"     : "Y",
            "NODE_LIMIT_EXCEEDED"  : "!",
            "SEARH-SPACE_EXHAUSTED": "x"}

print("\n                **TESTS SUMMARY**\n")

print("     Test        #max   Result   #gen      #inQ     Time s  cost")
for i, test in enumerate(TEST_RESULTS):
    max  = TEST_RESULTS[test]['args']['max_nodes']
    tc  = TEST_RESULTS[test]['result']['termination_condition']
    stc = short_tc[tc]
    
    ng  = TEST_RESULTS[test]['search_stats']['nodes_generated']
    nq  = TEST_RESULTS[test]['search_stats']['nodes_left_in_queue']
    time = round( TEST_RESULTS[test]['search_stats']['time_taken'], 2 )

    path = TEST_RESULTS[test]['result']['path']
    if isinstance(path, list):
      cost = len(path)
    else:
      cost = '!'
    # cost = len(path)
    print( f"{test:>10}:   {max:>8}    {stc}  {ng:>8} {nq:>8}     {time:>6}    {cost}")



                **TESTS SUMMARY**

     Test        #max   Result   #gen      #inQ     Time s  cost
    BFS_LT:     180000    !    469739     4087      33.05    !
    BFS_LF:    2000000    !   2000001  1287448     249.45    !
 DFS_LT_RF:      14000    Y     22530     6169       2.62    7031
 DFS_LT_RT:      80000    !    133721    32181      56.27    !
 DFS_LF_RF:      50000    !     50001    29999      24.78    !
 DFS_LF_RT:      50000    Y     35868    23178      47.69    12689
        h1:       1000    Y       923      220       0.11    63
        h2:        200    Y       279       65       0.07    47
      h1A*:     200000    Y    383506    18328      23.41    31
      h2A*:      30000    Y     48694     8520       2.28    31


# **Heuristic Function**

In [ ]:
def misplaced_tiles(state):
  count = 0
  for i in range(0, 3):
    for j in range(0, 3):
      if state[1][i][j] != self.goal_layout[i][j]:
        count += 1
  return count

In [ ]:
def manhattan(state):
  count = 0
  for i in range(0, 3):
    for j in range(0, 3):
      val = state[1][i][j]
      x, y = number_position_in_layout(val, self.goal_layout[i][j])
      count += abs(x - i)
      count += abs(y - j)
  return count